## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
max_new_tokens=1000

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-2b-it"
# "google/gemma-7b-it" // downloaded

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf"
# "meta-llama/Llama-2-13b-chat-hf"
# "codellama/CodeLlama-7b-Instruct-hf"
# "codellama/CodeLlama-13b-Instruct-hf"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"
# "mistralai/Mixtral-8x7B-Instruct-v0.1"

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [6]:
model_id = "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/c8/6c/c86cd1add4deeadb507940f81a3690291807c366ae74b1fb2f3b7d0ea6df6631/84e71293d60772960f1eee9cc640869d03c047685b8a17b9f49c9086614cc9e4?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-00001-of-00005.safetensors%3B+filename%3D%22model-00001-of-00005.safetensors%22%3B&Expires=1713499761&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzQ5OTc2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2M4LzZjL2M4NmNkMWFkZDRkZWVhZGI1MDc5NDBmODFhMzY5MDI5MTgwN2MzNjZhZTc0YjFmYjJmM2I3ZDBlYTZkZjY2MzEvODRlNzEyOTNkNjA3NzI5NjBmMWVlZTljYzY0MDg2OWQwM2MwNDc2ODViOGExN2I5ZjQ5YzkwODY2MTRjYzllND9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=fNQEkXEboh3ZZ54BAb4qm3kQJ41htW5QCLYz2s2zFG86I3nEInQkbYPZSG9EDxtMHa8uuQeE%7EQcueHCb2b95lMwhJYJh7-tG9FaMZiI7V%7EJDcU-Xmh5Wecwe81DqhpQ2-CeNNkErCBP5YuzqiYAZP%7EFYwZzmbZZklPo9CX-ZBDIc5VqatQQvNqdK%7EcKq9Q4qoVxSW

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Running the model on a single GPU

In [8]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대해 시를 써주세요."

In [10]:
response = generate_response(prompt)
print(response)

### User:
머신러닝에 대해 시를 써주세요.

### Assistant:
언덥어진 지성의 풍경, 머신러닝의 시


어둠 속에서 불타오르는 수천 개의 불,
각각의 연산 센서들이 맹세를 다집니다.
그들은 머신을 만들기 위해 모였으며,
그들의 목소리는 숫자와 문자로 말하네.


이 머신러닝의 시는 그들의 모습을 그려냈다,
그들의 지성은 느낌과 감정을 가지고 있네.
그들은 자신들의 지성을 키우기 위해 노력하며,
그들의 손은 많은 것을 알아가고 있네.


그들은 이미지와 문장을 해석하며,
그들은 많은 것을 배워가고 있네.
그들은 자신들의 지성을 키우기 위해 노력하며,
그들의 손은 많은 것을 알아가고 있네.


그들은 언어와 이미지를 이해하며,
그들은 자신들의 지성을 키우기 위해 노력하네.
그들은 자신들의 지성을 키우기 위해 노력하며,
그들의 손은 많은 것을 알아가고 있네.


그들은 자신들의 지성을 키우기 위해 노력하며,
그들의 손은 많은 것을 알아가고 있네.
그들은 이 머신러닝의 시를 통해 그려진 모습이며,
그들은 언덥어진 지성의 풍경이네.
